In [62]:
from sklearn.ensemble import RandomForestRegressor as rforest
from sklearn.tree import DecisionTreeRegressor as dtr
from sklearn.neighbors import KNeighborsRegressor as knn
from sklearn import tree
from PIL import Image
import numpy as np
from IPython.display import Image as display
import graphviz 
from copy import deepcopy

## a.)
Start with an image of the Mona Lisa. If you don’t like the Mona Lisa, pick another interesting
image of your choice.

In [63]:
im = Image.open("mona_lisa.jpg")
pix = np.array(im.getdata(),dtype=np.uint8)
w, h = im.size
pix.shape = (h, w, len(pix[0]))
print(pix.shape)

(900, 604, 3)


## b.)
To build your “training set,” uniformly sample 5,000 random (x, y)
coordinate locations.

Done

What other preprocessing steps are necessary for random forests inputs? Describe them,
implement them, and justify your decisions. In particular, do you need to perform mean
subtraction, standardization, or unit-normalization?

None

In [64]:
sampleset = []
samplecoords = []
for sample in range(5000):
    row = np.random.randint(0, h-1)
    col = np.random.randint(0, w-1)
    newcoords = [row, col] 
    sampleset.append(pix[row,col])
    samplecoords.append(newcoords)
np.asarray(sampleset)
np.asarray(samplecoords)

array([[542, 211],
       [430, 441],
       [190, 374],
       ..., 
       [ 89, 433],
       [849, 271],
       [546, 569]])

In [65]:
# img = Image.fromarray(pix, 'RGB')
# img.show()

## c.)
Sample pixel values at each of the given coordinate locations.
Each pixel contains red, green, and blue intensity values, so decide how you want to handle
this.

I chose to regress all three values at once. I.e., Function maps (x, y) coordinates to (r, g , b) values.

What other preprocessing steps are necessary for random regression forest outputs? Describe
them, implement them, and justify your decisions.

None

## d.)

To build the final image, for each pixel of the output, feed the pixel coordinate through the
random forest and color the resulting pixel with the output prediction. You can then use
imshow to view the result. (If you are using grayscale, try imshow(Y, cmap=’gray’) to avoid
fake-coloring).

Implementation Used: sklearn.ensemble.RandomForestRegressor <br>
CITE: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [66]:
regr = rforest(max_depth=7, random_state=0)
regr.fit(samplecoords, sampleset)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
           verbose=0, warm_start=False)

In [67]:
# newpix = np.zeros(shape=(h,w,3))
# for row in range(h):
#     for col in range(w):
#         newpix[row,col] = regr.predict(np.array([row,col]))

In [68]:
testcoords = []
for row in range(h):
    for col in range(w):
        testcoords.append([row, col])
testcoords = np.asarray(testcoords)
out = regr.predict(testcoords)
out.shape = (h, w, 3)
out = out.astype(np.uint8)
img = Image.fromarray(out, 'RGB')
img.save("depth_7_10_trees.jpg")

<img src="depth_7_10_trees.jpg" style="width: 200px;"/>

## e.) Experimentation

#### i.)
Repeat the experiment for a random forest containing a single decision tree, but with
depths 1, 2, 3, 5, 10, and 15. How does depth impact the result? Describe in detail why.

With increased depth, the number of rectangles increases. The resulting image has a more granular and accurate appearance. The reason for this is that by increasing the number of layers, we have increased the number of split points that model the painting.

In [69]:
for depth in [1, 2, 3, 5, 10, 15]:
    regr = rforest(n_estimators = 1, max_depth=depth, random_state=0)
    regr.fit(samplecoords, sampleset)
    testcoords = []
    for row in range(h):
        for col in range(w):
            testcoords.append([row, col])
    testcoords = np.asarray(testcoords)
    out = regr.predict(testcoords)
    out.shape = (h, w, 3)
    out = out.astype(np.uint8)
    img = Image.fromarray(out, 'RGB')
    img.save("depth_"+str(depth)+"_1_tree.jpg")

<img src="depth_1_1_tree.jpg" style="width: 200px;"/>
<img src="depth_2_1_tree.jpg" style="width: 200px;"/>
<img src="depth_3_1_tree.jpg" style="width: 200px;"/>
<img src="depth_5_1_tree.jpg" style="width: 200px;"/>
<img src="depth_10_1_tree.jpg" style="width: 200px;"/>
<img src="depth_15_1_tree.jpg" style="width: 200px;"/>

#### ii.)
Repeat the experiment for a random forest of depth 7, but with number of trees equal to
1, 3, 5, 10, and 100. How does the number of trees impact the result? Describe in detail
why.

Increasing the number of trees in the forest makes the resulting image appear more smooth. The colors are more blended. The reason for this is that the pixel colors are now chosen as a majority vote among the leaves in the trees of the forests, and not all forests are trained on the same subsample of the training set. As such, increasing the number of trees in the forest increases the color accuracy at a given point.

In [70]:
for estimators in [1, 3, 5, 10, 100]:
    regr = rforest(n_estimators = estimators, max_depth=7, random_state=0)
    regr.fit(samplecoords, sampleset)
    testcoords = []
    for row in range(h):
        for col in range(w):
            testcoords.append([row, col])
    testcoords = np.asarray(testcoords)
    out = regr.predict(testcoords)
    out.shape = (h, w, 3)
    out = out.astype(np.uint8)
    img = Image.fromarray(out, 'RGB')
    img.save("depth_7_"+str(estimators)+"_trees.jpg")

<img src="depth_7_1_trees.jpg" style="width: 200px;"/>
<img src="depth_7_3_trees.jpg" style="width: 200px;"/>
<img src="depth_7_5_trees.jpg" style="width: 200px;"/>
<img src="depth_7_10_trees.jpg" style="width: 200px;"/>
<img src="depth_7_100_trees.jpg" style="width: 200px;"/>

#### iii.)
As a simple baseline, repeat the experiment using a k-NN regressor, for k = 1. This means
that every pixel in the output will equal the nearest pixel from the “training set.” Compare
and contrast the outlook: why does this look the way it does?

The resulting image looks comparable to the high-depth tree regression, but better resembles the Mona Lisa. The reason for this is that the pixel regions are more circular (rather than rectangular), which better supports the way that the human eye processes images.

In [71]:
neigh = knn(n_neighbors=1)
neigh.fit(samplecoords, sampleset) 
testcoords = []
for row in range(h):
    for col in range(w):
        testcoords.append([row, col])
testcoords = np.asarray(testcoords)
out = neigh.predict(testcoords)
out.shape = (h, w, 3)
out = out.astype(np.uint8)
img = Image.fromarray(out, 'RGB')
img.save("knn_k_1.jpg")

<img src="knn_k_1.jpg" style="width: 200px;"/>

#### iv.) 
Experiment with different pruning strategies of your choice. <br>
From Wikipedia <https://en.wikipedia.org/wiki/Pruning_(decision_trees)>
<img src="pruning_methods.png"/>

We attempted a form of reduced error pruning. We proceeded by doing a depth-first traversal of the graph. For each node traversed, we evaluate the score of the decision tree if the subgraph starting at the current node is pruned. If the score improves, we prune it.

In [72]:
regr = rforest(n_estimators = 1, max_depth=7, random_state=0)
regr.fit(samplecoords, sampleset)
rtree = regr.estimators_[0].tree_

In [73]:
testset = []
testcoords = []
for row in range(h):
    for col in range(w):
        newcoords = [row, col] 
        testset.append(pix[row,col])
        testcoords.append(newcoords)
np.asarray(testset)
np.asarray(testcoords)

array([[  0,   0],
       [  0,   1],
       [  0,   2],
       ..., 
       [899, 601],
       [899, 602],
       [899, 603]])

In [74]:
out = regr.predict(testcoords)
out.shape = (h, w, 3)
out = out.astype(np.uint8)
img = Image.fromarray(out, 'RGB')
img.save("pre-prune_test.png")

In [75]:
dot_data = tree.export_graphviz(rtree, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("monalisa", view=False) 

'monalisa.pdf'

In [76]:
# Returns feature and indices of left and right children
def children(tree, index):
    root = tree.feature[index]
    left_index = tree.children_left[index]
    right_index = tree.children_right[index]
    if(right_index > index and left_index > index):
        return (left_index, right_index)
    else:
        return None
pruneint = children(rtree, 0)[1]

In [77]:
def get_subtree_nodes(tree, index, removal):
    removal.append(index)
    nodechildren = children(tree, index)
    if(nodechildren != None):
        get_subtree_nodes(tree, nodechildren[0], removal)
        get_subtree_nodes(tree, nodechildren[1], removal)
    return removal

def prune_subtree(tree, index):
    new_tree = deepcopy(tree)
    removal = get_subtree_nodes(new_tree, index, [])
    newval = new_tree.value[index]
    for i in removal:
        new_tree.children_left[i] = -1
        new_tree.children_right[i] = -1
        new_tree.value[i] = newval
#     new_tree.node_count -= len(removal)
#     new_tree.node_count += 1
    return new_tree

new_tree = prune_subtree(rtree, pruneint)
dot_data = tree.export_graphviz(new_tree, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("prune", view=False) 

'prune.pdf'

In [78]:
regr2 = rforest(n_estimators = 1, max_depth=7, random_state=0)
regr2.fit(samplecoords, sampleset)
regr2.estimators_[0].tree_ = new_tree
print(regr.score(testcoords, testset) - regr2.score(testcoords, testset))
out = regr2.predict(testcoords)
out.shape = (h, w, 3)
out = out.astype(np.uint8)
img = Image.fromarray(out, 'RGB')
img.save("prune_test.png")

0.139878324665


In [79]:
def reduced_error_pruning(regressor, startindex, testcoords, testset):
    print("PRUNE:", startindex)
    current_score = regressor.score(testcoords, testset)
    regrtree = regressor.estimators_[0].tree_
    nodechildren = children(regrtree, startindex)
    newregr = deepcopy(regressor)
    if(nodechildren != None):
        left_tree = prune_subtree(regrtree, nodechildren[0])
        right_tree = prune_subtree(regrtree, nodechildren[1])
        joint_tree = prune_subtree(left_tree, nodechildren[1])
        print("SCORING")
        newregr.estimators_[0].tree_ = left_tree
        left_score = newregr.score(testcoords, testset)
        newregr.estimators_[0].tree_ = right_tree
        right_score = newregr.score(testcoords, testset)
        if(left_score > current_score and right_score > current_score):
            print("PRUNING LEFT AND RIGHT CHILDREN OF", startindex)
            regressor.estimators_[0].tree_ = joint_tree
        elif(left_score > current_score):
            print("PRUNING LEFT CHILDREN OF", startindex)
            regressor.estimators_[0].tree_ = left_tree
            reduced_error_pruning(regressor, nodechildren[1], testcoords, testset)
        elif(right_score > current_score):
            print("PRUNING RIGHT CHILDREN OF", startindex)
            regressor.estimators_[0].tree_ = right_tree
            reduced_error_pruning(regressor, nodechildren[0], testcoords, testset)
        else:
            print("NO ERROR REDUCTION", startindex)
            reduced_error_pruning(regressor, nodechildren[0], testcoords, testset)
            reduced_error_pruning(regressor, nodechildren[1], testcoords, testset)
    return regressor

In [80]:
newregressor = reduced_error_pruning(regr, 0, testcoords, testset)
out = newregressor.predict(testcoords)
out.shape = (h, w, 3)
out = out.astype(np.uint8)
img = Image.fromarray(out, 'RGB')
img.save("red_prune_test.png")

PRUNE: 0
NO ERROR REDUCTION 0
PRUNE: 1
NO ERROR REDUCTION 1
PRUNE: 2
NO ERROR REDUCTION 2
PRUNE: 3
PRUNING LEFT CHILDREN OF 3
PRUNE: 19
PRUNING RIGHT CHILDREN OF 19
PRUNE: 20
NO ERROR REDUCTION 20
PRUNE: 21
NO ERROR REDUCTION 21
PRUNE: 22
PRUNE: 23
PRUNE: 24
PRUNE: 30
NO ERROR REDUCTION 30
PRUNE: 31
NO ERROR REDUCTION 31
PRUNE: 32
NO ERROR REDUCTION 32
PRUNE: 33
NO ERROR REDUCTION 33
PRUNE: 34
PRUNE: 35
PRUNE: 36
NO ERROR REDUCTION 36
PRUNE: 37
PRUNE: 38
PRUNE: 39
NO ERROR REDUCTION 39
PRUNE: 40
NO ERROR REDUCTION 40
PRUNE: 41
PRUNE: 42
PRUNE: 43
NO ERROR REDUCTION 43
PRUNE: 44
PRUNE: 45
PRUNE: 46
NO ERROR REDUCTION 46
PRUNE: 47
PRUNING LEFT CHILDREN OF 47
PRUNE: 51
NO ERROR REDUCTION 51
PRUNE: 52
PRUNE: 53
PRUNE: 54
NO ERROR REDUCTION 54
PRUNE: 55
NO ERROR REDUCTION 55
PRUNE: 56
PRUNE: 57
PRUNE: 58
NO ERROR REDUCTION 58
PRUNE: 59
PRUNE: 60
PRUNE: 61
NO ERROR REDUCTION 61
PRUNE: 62
NO ERROR REDUCTION 62
PRUNE: 63
NO ERROR REDUCTION 63
PRUNE: 64
NO ERROR REDUCTION 64
PRUNE: 65
NO ERROR 

In [82]:
print("Score Difference:", newregressor.score(testcoords, testset) - regr2.score(testcoords, testset))
print("New Score:", newregressor.score(testcoords, testset)) 
print("Old Score:", regr2.score(testcoords, testset))

Score Difference: 0.141300720629
New Score: 0.763080636352
Old Score: 0.621779915724


1 Tree, 7 Layers
<img src="pre-prune_test.png" style="width: 200px;"/>
After Reduced Error Pruning
<img src="red_prune_test.png" style="width: 200px;"/>

In [84]:
dot_data = tree.export_graphviz(newregressor.estimators_[0].tree_, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("pruned", view=False) 

'pruned.pdf'

## f. Analysis.

#### i. 
What is the decision rule at each split point? Write down the 1-line formula for the split
point at the root node for one the trained decision trees inside the forest. Feel free to
define any variables you need. <br>

I DONT KNOW

#### ii. 
Why does the resulting image look like the way it does? What shape are the patches of
color, and how are they arranged? <br>

The image looks boxy because the decision tree splits the values by discrete thresholds.
The patches of color are rectangular. They are arranged around the randomly sampled training set of 5000 points.

#### iii. 
Straightforward: How many patches of color may be in the resulting image if the forest
contains a single decision tree? Define any variables you need. <br>

There may be up to |x| patches of color, where x is the training set.

#### iv. 
Tricky: How many patches of color might be in the resulting image if the forest contains
n decision trees? Define any variables you need.

The color of a specific pixel is decided by majority vote of the trees in the forest.
Given that there are n trees and each tree was trained on a different subset y_n of the sample set, x. 
There can only be |y_n|*|n| patches of color.